In [34]:
## importing jupyter notebook as a module: https://stackoverflow.com/questions/20186344/ipynb-import-another-ipynb-file
%run tracker.py.ipynb
import numpy as np
import cv2
import pickle
import glob

In [35]:
# https://wayhome25.github.io/cs/2017/04/04/cs-04/
# https://stackoverflow.com/questions/13906623/using-pickle-dump-typeerror-must-be-str-not-bytes
'''
def pickle_dump(dict_name,file_name):
    f = open(file_name,'wb')
    pickle.dump(dict_name,f)
    f.close()

import pickle
mailbox = {'sarah':'lee','seok':'lee'}
pickle_dump(mailbox,'dict.txt')    
    
'''

dist_pickle = pickle.load(open('calibration_pickle.p','rb'))


In [36]:
# checking data type and the contents of data
print(dist_pickle)
type(dist_pickle)

{'dist': array([[-0.24688775, -0.02373133, -0.00109842,  0.00035108, -0.00258571]]), 'mtx': array([[  1.15777930e+03,   0.00000000e+00,   6.67111054e+02],
       [  0.00000000e+00,   1.15282291e+03,   3.86128938e+02],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])}


dict

In [37]:
# how to extract data from dict data type in python:
# https://stackoverflow.com/questions/5404665/accessing-elements-of-python-dictionary-by-index
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

In [38]:
# play list number 14:27
# https://www.youtube.com/watch?v=vWY8YUayf9Q&feature=youtu.be
    

In [46]:
# https://classroom.udacity.com/nanodegrees/nd013-ent/parts/f114da49-70ce-4ebe-b88c-e0ab576aed25/modules/37588c52-f496-4568-898e-651cbd0e8fc1/lessons/4883b3e6-1679-48d3-8fa3-b5893659d657/concepts/e6115672-155d-4c10-b640-fe20a4f4b0a6
# sobel extraction:
'''
https://blog.naver.com/roboholic84/220482877717
https://iskim3068.tistory.com/49 # sobel,kenny, prewitt, robert, etc. 
https://076923.github.io/posts/Python-opencv-14/


sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, 3)
cv2.Sobel(grayscale image, accuracy, x direction derivative, y direction derivative, kernel size) 
cv2.Sobel(그레이스케일 이미지, 정밀도, x방향 미분, y방향 미분, 커널, 배율, 델타, 픽셀 외삽법)를 이용하여 가장자리 검출을 적용합니다.

정밀도는 결과 이미지의 이미지 정밀도를 의미합니다. 정밀도에 따라 결과물이 달라질 수 있습니다.

x 방향 미분은 이미지에서 x 방향으로 미분할 값을 설정합니다.

y 방향 미분은 이미지에서 y 방향으로 미분할 값을 설정합니다.

커널은 소벨 커널의 크기를 설정합니다. 1, 3, 5, 7의 값을 사용합니다.

배율은 계산된 미분 값에 대한 배율값입니다.

'''
def abs_sobel_thresh(img,orient='x',sobel_kernel=3,thresh=(0,255)):
    # calculate directional gradient
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    if orient =='x':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel))
# normalization     
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
# numpy.zeros_like is the same as zeros in matlab: https://www.geeksforgeeks.org/numpy-zeros_like-python/
    binary_output = np.zeros_like(scaled_sobel)
# if scaled_sobel>=0 and scaled_sobel<=255, then set to 1    
    binary_output[(scaled_sobel>=thresh[0])&(scaled_sobel<=thresh[1])] = 1
    return binary_output


In [47]:
# maginitude based gradient check
# https://classroom.udacity.com/nanodegrees/nd013-ent/parts/f114da49-70ce-4ebe-b88c-e0ab576aed25/modules/37588c52-f496-4568-898e-651cbd0e8fc1/lessons/4883b3e6-1679-48d3-8fa3-b5893659d657/concepts/78be8208-1af0-4cb4-8a60-2d904b696449
    
def mag_thresh(image,sobel_kernel = 3,mag_thresh =(0,255)):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    gradmag = np.sqrt(sobelx**2,sobely**2)
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

In [48]:
# directional gradient check:
# https://classroom.udacity.com/nanodegrees/nd013-ent/parts/f114da49-70ce-4ebe-b88c-e0ab576aed25/modules/37588c52-f496-4568-898e-651cbd0e8fc1/lessons/4883b3e6-1679-48d3-8fa3-b5893659d657/concepts/d9ad4008-6744-4dc1-803d-e43a67cc050f

def dir_threshold(image, sobel_kernel = 3, thresh = (0,np.pi/2)):
    # calculate gradient direction
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    with np.errstate(divide='ignore',invalid='ignore'):
        absgraddir = np.absolute(np.arctan(sobely/sobelx))
        binary_output = np.zeros_like(absgraddir)
        #apply threshold
        binary_output[(absgraddir >= thresh[0])&(absgraddir <= thresh[1])]=1
    return binary_output
    
    

In [49]:
## colar threshold:
## https://classroom.udacity.com/nanodegrees/nd013-ent/parts/f114da49-70ce-4ebe-b88c-e0ab576aed25/modules/37588c52-f496-4568-898e-651cbd0e8fc1/lessons/4883b3e6-1679-48d3-8fa3-b5893659d657/concepts/d7542ed8-36ce-4407-bd0a-4a38d17d2325
    
def color_threshold(image,sthresh = (0,255),vthresh = (0,255)):
    hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0])&(s_channel <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >=vthresh[0])&(v_channel <= vthresh[1])] = 1
    
    output = np.zeros_like(s_channel)
    output[(s_binary ==1)&(v_binary ==1)] = 1
    return output



In [50]:
def window_mask(width,height,img_ref,center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width)):min(int(center+width),img_ref.shape[1])] = 1
    return output

In [51]:
## make a list of test images
## https://wikidocs.net/83
'''
import glob
glob.glob('*.*')
['INSTALL.LOG', 'LICENSE.txt', 'NEWS.txt', 'py.ico', 'pyc.ico',
'pycon.ico', 'python.exe', 'pythonw.exe', 'readme.txt',
'UNWISE.EXE', 'w9xpopen.exe']
glob.glob('*.txt')
['LICENSE.txt', 'NEWS.txt', 'readme.txt']

'''

images = glob.glob('test_images/test*.jpg')


In [52]:
for idx,fname in enumerate(images):
    # read in image
    img = cv2.imread(fname)
    ksize = 3
    # undistort the image
    img = cv2.undistort(img,mtx,dist,None,mtx)
    
    
    preprocessImage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient = 'x', thresh =(12,255)) 
    grady = abs_sobel_thresh(img, orient = 'y', thresh =(25,255))
    c_binary = color_threshold(img, sthresh = (100,255), vthresh = (50,255))
    preprocessImage[((gradx == 1)&(grady == 1)|(c_binary == 1))] = 255
    
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(0, 255))
    dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0, np.pi/2))
    
    #preprocessImage[((gradx == 1)&(grady == 1)|(mag_binary == 1) & (dir_binary == 1)|(c_binary == 1))] = 255
    
    img_size =(img.shape[1],img.shape[0])
    bot_width = .76
    mid_width = .08
    height_pct= .62
    bottom_trim= .935
    
 
    offset = img_size[0]*.25
    src = np.float32([[img.shape[1]*(.5-mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(.5+mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(.5+bot_width/2),img.shape[0]*bottom_trim],[img.shape[1]*(.5-bot_width/2),img.shape[0]*bottom_trim]])
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0],[img_size[0]-offset, img_size[1]],[offset, img_size[1]]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)                 
    warped = cv2.warpPerspective(preprocessImage, M, img_size, flags=cv2.INTER_LINEAR)
    
    window_width = 25
    window_height = 80
    
    #set up the overall class to do all the tracking 
    '''
    https://classroom.udacity.com/nanodegrees/nd013-ent/parts/f114da49-70ce-4ebe-b88c-e0ab576aed25/modules/37588c52-f496-4568-898e-651cbd0e8fc1/lessons/150ec3c9-0005-4da4-99fb-d40780f7ccb6/concepts/2f928913-21f6-4611-9055-01744acc344f
    
    '''
    curve_centers = tracker(Mywindow_width = window_width, Mywindow_height = window_height, Mymargin = 25, My_ym = 37/720, My_xm = 3.7/640,Mysmooth_factor = 15)
    
    window_centroids = curve_centers.find_window_centroids(warped)
    
    
    # points used to draw all the left and right windows
    
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    # points used to find the left and right lanes
    
    rightx = []
    leftx = []
    
    
    # Go through each level and draw the windows
    
    for level in range(0,len(window_centroids)):
        
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])
        
        # window_mask is a function to draw window areas
        l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
        r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)
        
        # add center value found in frame to the list of lane points per left,right
        

        
        # add graphic points from window mask here to total pixels found
        
        l_points[(l_points == 255)|((l_mask == 1))] = 255
        r_points[(r_points == 255)|((r_mask == 1))] = 255
        
    
    #template = np.array(r_points+l_points,np.uint8)
    #zero_channel = np.zeros_like(template)
    #template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8)
    #warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    #result = cv2.addWeighted(warpage,1,template,0.5,0.0)
                            
    # result = warped
    
    
    yvals = range(0,warped.shape[0])
    
    res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)
    
    left_fit = np.polyfit(res_yvals,leftx,2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx,np.int32)
    
    
    right_fit = np.polyfit(res_yvals,rightx,2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    right_fitx = np.array(right_fitx,np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2),axis= 0),np.concatenate((yvals,yvals[::-1]),axis = 0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis= 0),np.concatenate((yvals,yvals[::-1]),axis = 0))),np.int32)
    inner_lane = np.array(list(zip(np.concatenate((left_fitx+window_width/2,right_fitx[::-1]-window_width/2),axis = 0),np.concatenate((yvals,yvals[::-1]),axis = 0))),np.int32)
  
    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    cv2.fillPoly(road,[left_lane],color = [255,0,0])
    cv2.fillPoly(road,[right_lane],color = [0,0,255])
    cv2.fillPoly(road,[inner_lane],color = [0,255,0])
    
    cv2.fillPoly(road_bkg,[left_lane],color=[255,255,255])
    cv2.fillPoly(road_bkg,[right_lane],color = [255,255,255])
    
    #cv2.fillPoly(road_bkg,[left_lane],color=[255,0,0])
    #cv2.fillPoly(road_bkg,[right_lane],color = [0,0,255])
    
    road_warped = cv2.warpPerspective(road,Minv,img_size,flags = cv2.INTER_LINEAR)
    road_warped_bkg = cv2.warpPerspective(road_bkg,Minv,img_size,flags = cv2.INTER_LINEAR)
     
    base = cv2.addWeighted(img,1.0,road_warped_bkg,-1.0,0.0)
    result = cv2.addWeighted(base,1.0,road_warped,0.7,0.0)
    
    
    
    ym_per_pix = curve_centers.ym_per_pix
    xm_per_pix = curve_centers.xm_per_pix
    
    curve_fit_cr = np.polyfit(np.array(res_yvals,np.float32)*ym_per_pix,np.array(leftx,np.float32)*xm_per_pix,2)
    curverad = ((1+(2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])
    
    #calculate the offset of the car on the road
    
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos= 'left'
    if center_diff <= 0:
        side_pos = 'right'

        
    # draw the text showing curvature, offset, and speed 
    
    cv2.putText(result,'Radius of Curvature = ' +str(round(curverad,3))+'(m)',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result,'Vehicle is '+str(abs(round(center_diff,3)))+'m '+ side_pos+' of center',(50,100) , cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    
        
        
                        
    write_name = 'test_images/final'+str(idx)+'.jpg'
    
    cv2.imwrite(write_name,result)